https://www.youtube.com/watch?v=qu2qAXr4Des&t=429s


Investigar metodo de autoguardado en el entrenamiento para no perder el avance.

✅ Estrategia para retomar el entrenamiento en Colab:

Guardar checkpoints: Usa **model.save()** o PyTorch/TensorFlow **torch.save()** para guardar el progreso cada ciertas épocas.
Reanudar el entrenamiento: Carga el último checkpoint en una nueva sesión y continúa desde ahí.
Dividir el entrenamiento: En lugar de 10 horas seguidas, entrena en bloques de 2-3 horas y guarda el progreso.

In [ ]:
#----------------------------------------------------------------------------------------------------

Para lograr 100 epochs en 3 horas, puedes optimizar el entrenamiento de **YOLO11n** en Google Colab con las siguientes estrategias:  

### 1️⃣ **Usar una instancia más potente (Colab Pro/Pro+)**  
Si estás en la versión gratuita, considera **Colab Pro o Pro+**, que te da acceso a GPUs más rápidas como **T4, P100 o A100** (cuando está disponible).  

### 2️⃣ **Aumentar el tamaño del batch (si tu GPU lo permite)**  
En tu configuración tienes `batch=16`, pero si tu GPU tiene suficiente memoria, intenta **batch=32 o más** para acelerar la convergencia.  

### 3️⃣ **Reducir el tamaño de la imagen (`imgsz`)**  
Actualmente usas `imgsz=640`, pero podrías probar **512 o incluso 416**, lo que reduce el tiempo por batch sin afectar demasiado la precisión.  

### 4️⃣ **Utilizar `cache` para evitar la sobrecarga de lectura de imágenes**  
Si las imágenes están almacenadas en disco, activar el cache acelera el acceso:  
```python
model.train(..., cache=True)
```

### 5️⃣ **Activar `multi-scale` para entrenar más rápido y robustecer el modelo**  
```python
model.train(..., multi_scale=True)
```

### 6️⃣ **Reducir el número de workers para evitar cuellos de botella en Colab**  
Google Colab usa menos núcleos para la carga de datos, prueba con:  
```python
model.train(..., workers=2)
```

### 7️⃣ **Guardar y reanudar el entrenamiento**  
Si se interrumpe el entrenamiento, puedes reanudarlo sin perder progreso usando el checkpoint más reciente:  
```python
model.train(resume=True)
```
Si Colab se desconecta, puedes usar:
```python
model = YOLO('/ruta/al/ultimo/pesos.pt')  
```
Y continuar desde ahí.

---

### 🚀 **Configuración optimizada**  
Prueba con estos parámetros:  
```python
model.train(
    data='/content/merge1-1/data.yaml',
    epochs=100,
    imgsz=512,  # Reducido para mayor velocidad
    batch=32,  # Ajustado si la GPU lo permite
    cache=True,  # Evita lectura repetida de imágenes
    multi_scale=True,  # Acelera y mejora la generalización
    workers=2,  # Ajustado para evitar cuellos de botella
    name="entrenamiento_epp",
    project="proyecto_epp"
)
```

Con estas optimizaciones, deberías lograr los **100 epochs en menos de 3 horas**. 🚀🔥

-------------------------------------------------------------------------------------
Sí, puedes reanudar el entrenamiento desde el último checkpoint sin perder el progreso anterior. Sigue estos pasos:  

---

### **🔄 Paso 1: Verifica si tienes un checkpoint guardado**
Cuando `YOLO` entrena, automáticamente guarda los pesos del modelo en la carpeta del proyecto. Busca la carpeta donde guardaste el entrenamiento, en tu caso:  

📂 **`/content/proyecto_epp/entrenamiento_epp/`**  

Dentro de esa carpeta, debería haber un archivo de pesos con un nombre similar a:  
- `last.pt` (último checkpoint guardado)  
- `best.pt` (mejor modelo según validación)  

---

### **🔄 Paso 2: Cargar el último checkpoint**
En tu nueva sesión de Colab, monta Google Drive (si lo usaste) o revisa si los archivos aún están en `/content/`. Luego, carga el modelo con:  

```python
from ultralytics import YOLO  

# Cargar el último checkpoint
model = YOLO('/content/proyecto_epp/entrenamiento_epp/weights/last.pt')  

# Reanudar el entrenamiento
model.train(resume=True)
```

---

### **🛠️ Explicación**
- `YOLO('/ruta/al/archivo.pt')` → Carga los pesos del último modelo.  
- `train(resume=True)` → Continúa desde donde quedó, manteniendo hiperparámetros y estadísticas de entrenamiento.  

Si el archivo `last.pt` no está en `/content/`, revisa en Google Drive si lo guardaste antes. Si no lo encuentras, lamentablemente, tendrás que reiniciar desde **epoch 0**.  

---

### **📌 Recomendación para el futuro**
Para evitar perder el progreso si Colab se cierra, guarda los pesos en Google Drive al finalizar cada sesión:  

```python
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/proyecto_epp /content/drive/MyDrive/
```

Así, la próxima vez solo copias de vuelta antes de reanudar:  

```python
!cp -r /content/drive/MyDrive/proyecto_epp /content/
```

¡Con esto, no volverás a perder tu progreso! 🚀🔥

------------------------------------------------------------------------------------------------

Te recomiendo agregar un bloque de código que verifique si ya existe el checkpoint (por ejemplo, el archivo `last.pt`) y, de ser así, cargar ese modelo para reanudar el entrenamiento. Puedes hacer esto justo después de la parte en la que listas los archivos del directorio de resultados. Aquí te dejo un ejemplo de cómo modificar tu código:

---

### Código modificado con reanudación de entrenamiento

```python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image

!pip install split-folders pillow ultralytics
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="w1vFv4CrKeWMlQiAC4ti")
project = rf.workspace("patcontrol").project("merge1-svxaj")
version = project.version(1)
dataset = version.download("yolov11")

# Specify the path to the file
file_path = '/content/merge1-1/data.yaml'       # Copiar y pegar ruta del archivo yaml en el archivo anteriormente descargado

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file at {file_path} exists.")
else:
    print(f"The file at {file_path} does not exist.")

# Get the current working directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# List all files and directories in the current directory
files = os.listdir(cwd)
print(f"Files and directories in '{cwd}': {files}")

from ultralytics import YOLO
# Inicialmente carga el modelo base
model = YOLO("yolo11n.pt")
model.info()

from ultralytics import settings
# Configurar la carpeta de datasets
settings.update({'datasets_dir': '/content/merge1-1'})    # Reemplazar ruta con ruta de nuestra carpeta descargada

# Entrenar el modelo (la primera vez)
# model.train(
#     data='/content/merge1-1/data.yaml',   # Reemplazar direccion de ruta del archivo .yaml
#     epochs=100,
#     imgsz=640,  
#     batch=16,  
#     name="entrenamiento_epp",          # Nombre del entrenamiento
#     project="proyecto_epp"             # Nombre del proyecto
# )

# Verificar si existe el checkpoint para reanudar el entrenamiento
checkpoint_path = "/content/proyecto_epp/entrenamiento_epp/weights/last.pt"
if os.path.exists(checkpoint_path):
    print("Checkpoint encontrado. Reanudando entrenamiento desde:", checkpoint_path)
    # Cargar el modelo desde el checkpoint
    model = YOLO(checkpoint_path)
    # Reanudar entrenamiento
    model.train(resume=True)
else:
    print("No se encontró checkpoint. Iniciando entrenamiento desde cero.")
    model.train(
        data='/content/merge1-1/data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name="entrenamiento_epp",
        project="proyecto_epp"
    )

# Listar archivos en el directorio de resultados
directory = "/content/proyecto_epp/entrenamiento_epp"   # Reemplazar ruta por carpeta donde se encuentran los resultados del entrenamiento
if os.path.exists(directory):
    print("Archivos en el directorio:")
    print(os.listdir(directory))
else:
    print(f"El directorio {directory} no existe.")
```

---

### Explicación de los cambios

1. **Bloque comentado del entrenamiento inicial:**  
   He dejado comentada la llamada original a `model.train(...)` para que puedas usarla la primera vez que inicies el entrenamiento. Luego, si se cierra Colab y vuelves a ejecutar el cuaderno, el bloque condicional detectará si existe el checkpoint y reanudará desde allí.

2. **Verificación del checkpoint:**  
   Se define la variable `checkpoint_path` con la ruta del archivo `last.pt` que se guarda automáticamente durante el entrenamiento.  
   ```python
   checkpoint_path = "/content/proyecto_epp/entrenamiento_epp/weights/last.pt"
   ```

3. **Carga y reanudación:**  
   Si el archivo existe, se carga el modelo usando ese checkpoint y se reanuda el entrenamiento con `model.train(resume=True)`.  
   ```python
   model = YOLO(checkpoint_path)
   model.train(resume=True)
   ```

4. **Caso sin checkpoint:**  
   Si no se encuentra el checkpoint, se inicia el entrenamiento desde cero con los parámetros originales.

---

### Uso en futuras sesiones

Cuando tu Colab se cierre, asegúrate de **guardar la carpeta de resultados** (por ejemplo, en Google Drive) para que el archivo `last.pt` esté disponible en la siguiente sesión. Puedes copiar la carpeta a Drive y luego copiarla de vuelta al directorio de Colab antes de reanudar.

Con estos cambios, la próxima vez que ejecutes el cuaderno, se verificará la existencia del checkpoint y se reanudará el entrenamiento en lugar de comenzar desde cero. ¡Éxito con tu proyecto!

------------------------------------------------------------------------------------------------------

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
!pip install split-folders pillow ultralytics
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="w1vFv4CrKeWMlQiAC4ti")
project = rf.workspace("patcontrol").project("merge1-svxaj")
version = project.version(1)
dataset = version.download("yolov11")









loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Specify the path to the file
file_path = '/content/merge1-1/data.yaml'       # Copiar y pegar ruta del archivo yaml en el archivo anteriormente descargado

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file at {file_path} exists.")
else:
    print(f"The file at {file_path} does not exist.")

The file at /content/merge1-1/data.yaml exists.


In [ ]:
# Get the current working directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# List all files and directories in the current directory
files = os.listdir(cwd)
print(f"Files and directories in '{cwd}': {files}")


Current working directory: /content
Files and directories in '/content': ['.config', 'yolo11n.pt', 'mi_carpeta_zip.zip', 'proyecto_epp', 'merge1-1', 'drive', 'sample_data']


In [ ]:
# dir_path = cwd + "/sample_data"
# for files in os.listdir(dir_path):
#   os.remove(dir_path + "/" + files)
# os.rmdir(dir_path)

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [ ]:
from ultralytics import settings
# View all settings
settings.update({'datasets_dir': '/content/merge1-1'})    #reemplazar ruta con ruta de nuestra carpeta descargada

In [ ]:
# os.chdir("/content")
# Entrenar el modelo
model.train(
    data='/content/merge1-1/data.yaml',   #reemplazar direccion de ruta del archivo .yaml
    epochs=100,
    imgsz=640,  # Actualmente usa imgsz=640, pero podrías probar 512 o incluso 416, lo que reduce el tiempo por batch sin afectar demasiado la precisión.
    batch=16,  # Ajusta según tu GPU 16 recomendable
    name="entrenamiento_epp",          #nombre del entrenamiento
    project="proyecto_epp"             #nombre del proyecto
)

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/merge1-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=proyecto_epp, name=entrenamiento_epp2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning /content/merge1-1/train/labels.cache... 11572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11572/11572 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/merge1-1/valid/labels.cache... 2392 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2392/2392 [00:00<?, ?it/s]


Plotting labels to proyecto_epp/entrenamiento_epp2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to proyecto_epp/entrenamiento_epp2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.373      5.337      2.115         86        640:   5%|▌         | 39/724 [10:03<2:56:45, 15.48s/it]


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.make_archive('/content/proyecto_epp/comprimido', 'zip', '/content')


In [ ]:
from google.colab import files
files.download('/content/proyecto_epp/comprimido.zip')


In [ ]:
import os

# Listar archivos en el directorio
directory = "/content/proyecto_epp/entrenamiento_epp"   #reemplazar ruta por carpeta donde se encuentran los resultados del entrenamiento
if os.path.exists(directory):
    print("Archivos en el directorio:")
    print(os.listdir(directory))
else:
    print(f"El directorio {directory} no existe.")

El directorio /content/proyecto_epp/entrenamiento_epp no existe.


In [ ]:
# Crear DataFrame de métricas de entrenamiento
df_metrics = pd.read_csv("/content/proyecto_placas/entrenamiento_placas/results.csv")   #agregar ruta de results.csv


# Mostrar algunas métricas
print(df_metrics.head(5))

print(df_metrics.index.to_list())
print(df_metrics.columns.to_list())

df_metrics.describe()

In [ ]:
print(df_metrics.columns.to_list())
# Graficar las métricas
plt.figure(figsize=(12, 6))
plt.plot(df_metrics[['epoch']], df_metrics[['train/cls_loss']], label='Train Loss')
plt.plot(df_metrics[['epoch']], df_metrics[['metrics/mAP50(B)']], label='Val mAP@0.5')
plt.plot(df_metrics[['epoch']], df_metrics[['metrics/mAP50-95(B)']], label='Val mAP@0.5:0.95')
plt.plot(df_metrics[['epoch']], df_metrics[['metrics/precision(B)']], label='Validation Presicion')
plt.xlabel('Época')
plt.ylabel('Valor')
plt.title('Métricas de Entrenamiento')
plt.legend()
plt.show()

In [ ]:
# Evaluar el modelo
results = model.val()

# Mostrar los resultados
print(results)

In [ ]:
# Cargar el modelo entrenado
# modelo_entrenado = YOLO("/content/yolov8n.pt")
modelo_entrenado = YOLO("/content/proyecto_placas/entrenamiento_placas/weights/best.pt")    # Reemplazar ruta de best.pt

# Realizar inferencia en una imagen
resultados = modelo_entrenado.predict(source="/content/Imagen de WhatsApp 2024-12-11 a las 10.00.37_a5daf3b0.jpg", save=True)   # Reemplazar por ruta imagen que nos sirva para hacer una inferencia

# Acceder a los resultados
for resultado in resultados:
    print(f"Predicciones para la imagen: {resultado.path}")

    for caja, conf, clase in zip(resultado.boxes.xyxy, resultado.boxes.conf, resultado.boxes.cls):
        print(f"Bounding Box: {caja} - Confianza: {conf} - Clase: {clase}")

In [ ]:
# Cargar y mostrar la imagen con las predicciones
img = Image.open("/content/runs/detect/predict/Imagen de WhatsApp 2024-12-11 a las 10.00.37_a5daf3b0.jpg")  # Reemplazar por ruta de acceso a imagen .jpg
plt.imshow(img)
plt.axis('off')  # Ocultar los ejes
plt.show()

In [ ]:
import shutil
from google.colab import files

# Especifica el nombre de la carpeta que deseas comprimir y descargar
folder_path = "/content/proyecto_placas"  # Ruta de la carpeta en Colab
output_filename = "brayan_placas_v4.zip"   # Nombre del archivo zip de salida

# Comprimir la carpeta en un archivo ZIP
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_path)

# Descargar el archivo ZIP
files.download(output_filename)
